In [1]:
include("src/Elise.jl");

In [2]:
Random.seed!(1234);

---

## Config

In [3]:
const cfg = Config()
dump(cfg)

Config
  dt: Float64 0.1
  E_exc: Float64 1.0
  E_inh: Float64 -1.0
  g_l: Float64 0.1
  g_den: Float64 2.0
  g_e: Float64 0.3
  g_i: Float64 0.9
  δ: Float64 25.0
  η: Float64 1.0e-5
  λ: Float64 0.6
  a: Float64 21.0
  b: Float64 0.18


---

In [4]:
W_init(N,M) = rand(Normal(0., 0.0005), N, M);
B_init(N_latent, N) = Float64.(rand(N_latent, N) .< 1.5/N);
τ_d_init(N) = rand(Uniform(1., 20.), N)
τ_s_init(N) = rand(Uniform(1., 20.), N)

τ_s_init (generic function with 1 method)

In [5]:
N_lat(N_out) = 2*N_out

N_lat (generic function with 1 method)

In [6]:
TM, NT = 200., 1000

(200.0, 1000)

In [7]:
bench = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_lat, W_init, B_init, τ_d_init, τ_s_init; n_test=25);

Progress: 100%|█████████████████████████████████████████| Time: 0:13:09


In [8]:
bench_fixed = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_lat, W_init, B_init, N->10. *ones(N), N->10. *ones(N); n_test=25);

In [9]:
bench_dout_1 = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_lat, W_init, (N_latent, N) -> Float64.(rand(N_latent, N) .< 1. /N), τ_d_init, τ_s_init; n_test=25);

In [10]:
bench_dout_2 = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_lat, W_init, (N_latent, N) -> Float64.(rand(N_latent, N) .< 2. /N), τ_d_init, τ_s_init; n_test=25);

In [11]:
bench_20 = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_out->20, W_init, B_init, τ_d_init, τ_s_init; n_test=25);

In [12]:
bench_50 = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_out->50, W_init, B_init, τ_d_init, τ_s_init; n_test=25);

In [13]:
bench_100 = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_out->100, W_init, B_init, τ_d_init, τ_s_init; n_test=25);

In [14]:
bench_200 = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_out->200, W_init, B_init, τ_d_init, τ_s_init; n_test=25);

In [15]:
bench_300 = benchmark(cfg::Config, 0., TM, "Well Tempered Clavier/Preludes/",
(r,t;θ=.5) -> sum((r.>θ).&(t.>θ)) / sum((r.>θ).|(t.>θ)), NT,
N_out->300, W_init, B_init, τ_d_init, τ_s_init; n_test=25);

In [16]:
using PrettyTables
data = hcat(collect(keys(bench)),
(k->bench[k]).(collect(keys(bench))),
(k->bench_fixed[k]).(collect(keys(bench))),
(k->bench_dout_1[k]).(collect(keys(bench))),
(k->bench_dout_2[k]).(collect(keys(bench))),
(k->bench_20[k]).(collect(keys(bench))),
(k->bench_50[k]).(collect(keys(bench))),
(k->bench_100[k]).(collect(keys(bench))),
(k->bench_200[k]).(collect(keys(bench))),
(k->bench_300[k]).(collect(keys(bench))),
)
pretty_table(HTML, data; header=["File", "Classic", "Fixed" ,"dout=1", "dout=2", "20 lat", "50 lat", "100 lat", "200 lat", "300 lat"])

File,Classic,Fixed,dout=1,dout=2,20 lat,50 lat,100 lat,200 lat,300 lat
Prelude8.mid,0.836466,0.59625,0.806391,0.737918,0.780075,0.658135,0.866419,0.61,0.831481
Prelude23.mid,0.7664,0.638066,0.742176,0.875,0.661538,0.763693,0.914474,0.888889,0.838762
Prelude19.mid,missing,missing,missing,missing,missing,missing,missing,missing,missing
Prelude21.mid,0.886574,0.959135,0.895981,0.963768,0.428312,0.892774,0.930456,0.956835,0.9
Prelude18.mid,0.904762,0.881129,0.769553,0.915408,0.80086,0.796586,0.938462,0.865217,0.912387
Prelude4.mid,0.945993,0.938671,0.93686,0.960276,0.840442,0.953125,0.942177,0.975567,0.972028
Prelude9.mid,0.967626,0.967684,0.904348,0.965889,0.850847,0.896014,0.851852,0.960503,0.965827
Prelude20.mid,missing,missing,missing,missing,missing,missing,missing,missing,missing
Prelude3.mid,0.790043,0.33871,0.321368,0.630435,0.704255,0.575472,0.910165,0.523723,0.690722
Prelude5.mid,0.86646,0.812883,0.853293,0.880734,0.0660661,0.723529,0.901587,0.471883,0.564103
